In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
X_train = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')
Id = X_test['Id']

In [3]:
X_train.shape

(1460, 81)

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.impute import SimpleImputer
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.linear_model import RidgeCV,ElasticNet
from sklearn.metrics.regression import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split,RandomizedSearchCV

In [5]:
X_train.dropna(subset = ['SalePrice'],axis = 0, inplace = True)
y_train = X_train['SalePrice']


In [6]:
#finding for columns with missing values
col_with_missing = []
for col in X_train.columns:
    if X_train[col].isnull().any():
        col_with_missing.append(col)


In [7]:
X_train = X_train.drop(['Id','SalePrice'] , axis = 1)
X_test = X_test.drop(['Id'],axis = 1)

In [8]:
cate_col = []
num_col = []

for col in X_train:
    if X_train[col].nunique() < 10 and X_train[col].dtype == 'object':
        cate_col.append(col)
    if X_train[col].dtype == 'int64' or X_train[col].dtype == 'float64':
        num_col.append(col)
        
final_col = num_col + cate_col

In [9]:
X_train = X_train[final_col]
X_test = X_test[final_col]

#add a binary column if column has missing data
for col in col_with_missing:
    X_train[col + " missing"] = X_train[col].isnull()
    X_test[col + " missing"] = X_test[col].isnull()

In [10]:
X_train.shape, X_test.shape

((1460, 95), (1459, 95))

In [11]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# to ensure that both test and train have same number of columns 
final_train , final_test = X_train.align(X_test,axis = 1, join= "left")

In [12]:
#using Imputer to replace NaN values with median
imp = SimpleImputer(strategy = 'median')
cate_train= imp.fit_transform(final_train)
cate_test = imp.transform(final_test)

In [13]:
y_train = np.log1p(y_train)

### RidgeCV

In [14]:
rid = RidgeCV(alphas=[0.1,0.01,0.001,0.005], cv = 6)
rid.fit(cate_train,y_train)

RidgeCV(alphas=array([0.1  , 0.01 , 0.001, 0.005]), cv=6)

In [15]:
rid.alpha_

0.001

In [16]:
rid_score = -cross_val_score(rid,cate_train,y_train,scoring = 'neg_mean_absolute_error',cv = 5)
np.mean(rid_score),np.std(rid_score)


(0.09242414245275443, 0.006367822301055959)

### ElasticNet

In [18]:
elastic = ElasticNet(random_state = 42,max_iter = 5000)
elastic.fit(cate_train, y_train)


ElasticNet(max_iter=5000, random_state=42)

In [19]:
params = {'alpha' :[0.1,0.001,0.01,0.005,0.05, 0.5]}

In [20]:
rand = RandomizedSearchCV(elastic,params, random_state = 42)
rand.fit(cate_train, y_train)

RandomizedSearchCV(estimator=ElasticNet(max_iter=5000, random_state=42),
                   param_distributions={'alpha': [0.1, 0.001, 0.01, 0.005, 0.05,
                                                  0.5]},
                   random_state=42)

In [21]:
rand.best_estimator_

ElasticNet(alpha=0.001, max_iter=5000, random_state=42)

In [22]:
ela_score = -cross_val_score(rand,cate_train,y_train,scoring = 'neg_mean_absolute_error',cv = 5)
np.mean(ela_score), np.std(ela_score)


(0.08944507922539038, 0.0039030736856855938)

In [23]:
pred2 = rand.predict(cate_test)
pred2 = np.expm1(pred2)
submit= pd.DataFrame()
submit['Id'] = Id 
submit['SalePrice'] = pred2
submit.head(20)
submit.to_csv('submission_logtransform_elasticnet.csv', index = False)




"\npred2 = rand.predict(cate_test)\npred2 = np.expm1(pred2)\nsubmit= pd.DataFrame()\nsubmit['Id'] = Id \nsubmit['SalePrice'] = pred2\nsubmit.head(20)\n#submit.to_csv('submission_logtransform_elasticnet.csv', index = False)\n"